## Практическое задание к уроку 8

1. Обучите любую модель классификации на датасете IRIS до применения PCA и после него. Сравните качество классификации по отложенной выборке.

In [196]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [197]:
iris = datasets.load_iris()
X = iris.data
X[:5]


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [198]:
y = iris.target
np.unique(y)

array([0, 1, 2])

In [199]:
def scaling(x):
    result = (x - x.mean(axis=0)) / x.std(axis=0)
    return result

In [200]:
X = scaling(X.astype(float))

In [201]:
X[:5]

array([[-0.90068117,  1.01900435, -1.34022653, -1.3154443 ],
       [-1.14301691, -0.13197948, -1.34022653, -1.3154443 ],
       [-1.38535265,  0.32841405, -1.39706395, -1.3154443 ],
       [-1.50652052,  0.09821729, -1.2833891 , -1.3154443 ],
       [-1.02184904,  1.24920112, -1.34022653, -1.3154443 ]])

In [202]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [203]:
KNN = KNeighborsClassifier(n_neighbors=3)
KNN.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [204]:
y_pred = KNN.predict(X_test)

In [205]:
def accuracy(pred, y):
    return (sum(pred == y) / len(y))

In [206]:
print(f'Точность KNN без PCA {accuracy(y_pred, y_test):.3f}')

Точность KNN без PCA 0.980


In [207]:
X2_train = np.copy(X_train)

In [208]:
# Найдём собственные векторы и собственные значения (англ. Eigenvalues)
 
covariance_matrix = X2_train.T.dot(X2_train)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения в порядке убывания:')
for i in eig_pairs:
    print(i)

Собственные значения в порядке убывания:
(277.5858101822737, array([ 0.54074196, -0.21846945,  0.584708  ,  0.56390228]))
(91.03139733688748, array([-0.35683369, -0.93374502,  0.00729599, -0.02714271]))
(17.032958987017924, array([-0.71248088,  0.25477394,  0.13770394,  0.63913916]))
(2.005028303294107, array([ 0.26951596, -0.12442499, -0.79943789,  0.52228213]))


In [209]:
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперь оценим кумулятивную, то есть накапливаемую, дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[71.60636924231159, 23.482568673335606, 4.393842573266527, 0.5172195110862756]
Кумулятивная доля дисперсии по компонентам 
[ 71.60636924  95.08893792  99.48278049 100.        ]


In [210]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack([eig_pairs[i][1].reshape(4,1) for i in range(2)])

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.54074196 -0.35683369]
 [-0.21846945 -0.93374502]
 [ 0.584708    0.00729599]
 [ 0.56390228 -0.02714271]]


In [211]:
Z = X2_train.dot(W)

In [212]:
Z[:5]

array([[ 0.20682537,  0.39838922],
       [ 2.79284557, -0.65668504],
       [ 0.33916913,  0.22077926],
       [-2.23508064, -0.60417041],
       [ 2.91795365, -0.26604411]])

In [213]:
KNN.fit(Z, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [214]:
y2_pred = KNN.predict(Z_test)

In [215]:
print(f'Точность KNN после PCA {accuracy(y2_pred, y_test):.3f}')

Точность KNN после PCA 0.920


2. Напишите свою реализацию метода главных компонент посредством сингулярного разложения с использованием функции numpy.linalg.svd().

In [216]:
u, s, vh = np.linalg.svd(X_train, full_matrices=True)


In [217]:
eig_values, eig_vectors = s, vh.T

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения в порядке убывания:')
for i in eig_pairs:
    print(i)

Собственные значения в порядке убывания:
(16.660906643465534, array([ 0.54074196, -0.21846945,  0.584708  ,  0.56390228]))
(9.541037539853171, array([-0.35683369, -0.93374502,  0.00729599, -0.02714271]))
(4.127100554507722, array([-0.71248088,  0.25477394,  0.13770394,  0.63913916]))
(1.4159902200559489, array([ 0.26951596, -0.12442499, -0.79943789,  0.52228213]))


In [218]:
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперь оценим кумулятивную, то есть накапливаемую, дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[52.48350384735861, 30.055211948929056, 13.000774955779193, 4.460509247933131]
Кумулятивная доля дисперсии по компонентам 
[ 52.48350385  82.5387158   95.53949075 100.        ]


In [219]:
W = np.hstack([eig_pairs[i][1].reshape(4,1) for i in range(2)])

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.54074196 -0.35683369]
 [-0.21846945 -0.93374502]
 [ 0.584708    0.00729599]
 [ 0.56390228 -0.02714271]]


Для сравнения 

In [220]:
vh

array([[ 0.54074196, -0.21846945,  0.584708  ,  0.56390228],
       [-0.35683369, -0.93374502,  0.00729599, -0.02714271],
       [-0.71248088,  0.25477394,  0.13770394,  0.63913916],
       [ 0.26951596, -0.12442499, -0.79943789,  0.52228213]])

In [221]:
vh[:2].T # == W
#Это и есть веса признаков. Первые две строчки потому что хотим оставить два признака

array([[ 0.54074196, -0.35683369],
       [-0.21846945, -0.93374502],
       [ 0.584708  ,  0.00729599],
       [ 0.56390228, -0.02714271]])

In [222]:
Z2 = X_train.dot(W)

In [223]:
KNN.fit(Z2, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [224]:
y3_pred = KNN.predict(Z_test)

In [225]:
print(f'Точность KNN после PCA через SVD {accuracy(y3_pred, y_test):.3f}')

Точность KNN после PCA через SVD 0.920
